In [1]:
import pandas as pd
import numpy as np


In [2]:
import os 
dir = r"C:\Users\HP\Downloads\proj\dataset\train\boxes_transcripts_labels"
list_items = os.listdir(dir)


In [5]:
dataframes=[]
for file in list_items:
    if( file.endswith('tsv')):
        path = os.path.join(dir ,file)
        df = pd.read_csv(path ,header=None,delimiter =',')
        selected_columns = df.iloc[: , [0,1,2,3,4,5,7]]
        dataframes.append(selected_columns)

combined_data = pd.concat(dataframes, ignore_index=True)

# Rename columns for clarity
combined_data.columns = ['start_index' ,'end_index' ,'top_left_x', 'top_left_y', 'bottom_right_x', 'bottom_right_y', 'label_text' ]

combined_data 
        
        

,start_index,end_index,top_left_x,top_left_y,bottom_right_x,bottom_right_y,label_text
0,33,33,215,4,227,21,OTHER
1,35,44,235,3,308,21,OTHER
2,46,51,311,3,349,20,OTHER
3,53,60,352,3,401,20,OTHER
4,62,67,404,3,457,21,OTHER
...,...,...,...,...,...,...,...
237611,2325,2327,365,1064,385,1079,OTHER
237612,2329,2334,388,1064,429,1079,OTHER
237613,2336,2338,451,1063,482,1078,OTHER
237614,2340,2342,485,1063,509,1078,OTHER


In [6]:
combined_data['label_text'].unique()

array(['OTHER', 'ssnOfEmployee', 'einEmployerIdentificationNumber',
       'box1WagesTipsAndOtherCompensations',
       'box2FederalIncomeTaxWithheld', 'employerName',
       'box3SocialSecurityWages', 'box4SocialSecurityTaxWithheld',
       'employerAddressStreet_name', 'employerAddressZip',
       'employerAddressCity', 'employerAddressState', 'employeeName',
       'taxYear', 'box16StateWagesTips', 'box17StateIncomeTax'],
      dtype=object)

In [7]:
attributes = [
    'OTHER', 'ssnOfEmployee', 'einEmployerIdentificationNumber',
    'box1WagesTipsAndOtherCompensations', 'box2FederalIncomeTaxWithheld', 
    'employerName', 'box3SocialSecurityWages', 'box4SocialSecurityTaxWithheld',
    'employerAddressStreet_name', 'employerAddressZip',
    'employerAddressCity', 'employerAddressState', 'employeeName',
    'taxYear', 'box16StateWagesTips', 'box17StateIncomeTax'
]

# Create the dictionary mapping
attribute_mapping = {attribute: idx for idx, attribute in enumerate(attributes)}

print(attribute_mapping)

{'OTHER': 0, 'ssnOfEmployee': 1, 'einEmployerIdentificationNumber': 2, 'box1WagesTipsAndOtherCompensations': 3, 'box2FederalIncomeTaxWithheld': 4, 'employerName': 5, 'box3SocialSecurityWages': 6, 'box4SocialSecurityTaxWithheld': 7, 'employerAddressStreet_name': 8, 'employerAddressZip': 9, 'employerAddressCity': 10, 'employerAddressState': 11, 'employeeName': 12, 'taxYear': 13, 'box16StateWagesTips': 14, 'box17StateIncomeTax': 15}


In [8]:
combined_data['label_text'].replace(attribute_mapping ,inplace=True)
combined_data

,start_index,end_index,top_left_x,top_left_y,bottom_right_x,bottom_right_y,label_text
0,33,33,215,4,227,21,0
1,35,44,235,3,308,21,0
2,46,51,311,3,349,20,0
3,53,60,352,3,401,20,0
4,62,67,404,3,457,21,0
...,...,...,...,...,...,...,...
237611,2325,2327,365,1064,385,1079,0
237612,2329,2334,388,1064,429,1079,0
237613,2336,2338,451,1063,482,1078,0
237614,2340,2342,485,1063,509,1078,0


In [9]:
X= combined_data[['start_index' ,'end_index','top_left_x', 'top_left_y', 'bottom_right_x', 'bottom_right_y' ]].values

# Labels: Column 'class_label'
Y = combined_data['label_text'].values

In [10]:
X

array([[  33,   33,  215,    4,  227,   21],
       [  35,   44,  235,    3,  308,   21],
       [  46,   51,  311,    3,  349,   20],
       ...,
       [2336, 2338,  451, 1063,  482, 1078],
       [2340, 2342,  485, 1063,  509, 1078],
       [2344, 2352,  511, 1063,  572, 1078]], dtype=int64)

In [12]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [13]:
dtrain = xgb.DMatrix(X ,label=Y)

In [15]:
params = {
    "objective": "multi:softmax",  # Classification problem with 16 classes
    "num_class": 16,              # Number of classes
    "max_depth": 10,              # Tree depth; 10 is reasonable for your dataset size
    "eta": 0.3,                   # Learning rate; 0.3 is the default for faster convergence
    "subsample": 0.8,             # Sampling ratio for rows; avoids overfitting
    "colsample_bytree": 0.8,      # Sampling ratio for features per tree
    "gamma": 0,                   # No additional regularization
    "lambda": 1,                  # L2 regularization
    "alpha": 0                    # No L1 regularization
}


In [17]:
# Train the model
model = xgb.train(params, dtrain, num_boost_round=100)


### Testing the model 

In [18]:
import os

directory = r'C:\Users\HP\Downloads\proj\dataset\val_w_ann\boxes_transcripts_labels'
test_list_items = os.listdir(directory)
tsv_files=[]
for item in test_list_items:
    if item.endswith('.tsv'):
        full_path = os.path.join(directory,item)
        tsv_files.append(full_path)

# print(tsv_files)
dataframes =[]
for file in tsv_files:
    df = pd.read_csv(file ,header =None, delimiter =',')
    selected_columns = df.iloc[: , [0,1,2,3,4,5,7]]
    dataframes.append(selected_columns)

combined_data_test = pd.concat(dataframes, ignore_index=True)

# Rename columns for clarity
combined_data_test.columns = ['start_index' ,'end_index' ,'top_left_x', 'top_left_y', 'bottom_right_x', 'bottom_right_y', 'label_text' ]

combined_data_test

,start_index,end_index,top_left_x,top_left_y,bottom_right_x,bottom_right_y,label_text
0,33,33,216,22,225,36,OTHER
1,35,44,235,21,309,36,OTHER
2,46,51,311,20,350,35,OTHER
3,53,60,353,19,402,34,OTHER
4,62,67,405,19,456,33,OTHER
...,...,...,...,...,...,...,...
80448,2335,2337,364,1064,386,1078,OTHER
80449,2339,2344,389,1063,430,1078,OTHER
80450,2346,2348,451,1063,482,1077,OTHER
80451,2350,2352,484,1064,508,1077,OTHER


In [19]:
combined_data_test['label_text'].replace(attribute_mapping , inplace=True)
combined_data_test

,start_index,end_index,top_left_x,top_left_y,bottom_right_x,bottom_right_y,label_text
0,33,33,216,22,225,36,0
1,35,44,235,21,309,36,0
2,46,51,311,20,350,35,0
3,53,60,353,19,402,34,0
4,62,67,405,19,456,33,0
...,...,...,...,...,...,...,...
80448,2335,2337,364,1064,386,1078,0
80449,2339,2344,389,1063,430,1078,0
80450,2346,2348,451,1063,482,1077,0
80451,2350,2352,484,1064,508,1077,0


In [23]:
Xtest= combined_data_test.iloc[: ,0:6].values
Ytest =combined_data_test.iloc[: ,6].values

dtest = xgb.DMatrix(Xtest)


In [24]:

predictions =model.predict(dtest)
accuracy_score(Ytest ,predictions)

0.9928653996743441

In [25]:
predictions

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

### Tuning the parameters 

In [27]:
from scipy.stats import uniform, randint  # Distributions for random sampling
from sklearn.model_selection import RandomizedSearchCV

In [28]:

# Initialize the XGBoost classifier
model2 = xgb.XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")

# Define the parameter distributions
param_distributions = {
    "learning_rate": uniform(0.01, 0.3),  # Random values between 0.01 and 0.3
    "max_depth": randint(3, 10),          # Random integer values between 3 and 10
    "n_estimators": randint(50, 200),     # Random integer values between 50 and 200
    "subsample": uniform(0.6, 0.4),       # Random values between 0.6 and 1.0
    "colsample_bytree": uniform(0.6, 0.4),# Random values between 0.6 and 1.0
}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model2,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=3,       # Cross-validation folds
    scoring="accuracy",
    verbose=2,
    random_state=42,
    n_jobs=-1   # Use all CPU cores
)

# Fit the random search
random_search.fit(X, Y)
# Print the best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


C:\Users\HP\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:43:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'colsample_bytree': 0.9757995766256756, 'learning_rate': 0.27844820512829466, 'max_depth': 8, 'n_estimators': 144, 'subsample': 0.9687496940092467}
Best Score: 0.9854807871816519
